In [1]:
import os
import requests
import zipfile
import subprocess
import re
from datetime import datetime
from google.colab import auth

def setup_environment():
    """
    Authenticates the user and configures the necessary GCP project and bucket names.

    This function handles the initial setup by authenticating the user's Google account
    for use within the Colab environment. It then programmatically determines the
    current GCP Project ID. If it cannot be determined automatically, it will prompt
    the user to enter it manually. It also prompts the user for a base bucket name
    and combines it with the project ID to ensure global uniqueness.

    Returns:
        tuple: A tuple containing the Project ID and the generated GCS bucket name.
               Returns (None, None) if the project ID or bucket name is not provided.
    """
    print("Authenticating your Google account...")
    # This command will trigger a pop-up to authenticate your Google account.
    auth.authenticate_user()
    print("✅ Authentication successful.")

    project_id = ""
    try:
        # This command runs the gcloud CLI to get the currently configured project ID.
        project_id_process = subprocess.run(
            ["gcloud", "config", "get-value", "project"],
            capture_output=True, text=True, check=True
        )
        project_id = project_id_process.stdout.strip()
    except (subprocess.CalledProcessError, FileNotFoundError):
        # This block will run if gcloud is not configured or not found.
        pass

    if not project_id:
        print("⚠️ Could not automatically determine GCP Project ID.")
        project_id = input("Please enter your GCP Project ID: ").strip() # Added .strip()

    if not project_id:
        print("🔴 ERROR: Project ID is required to continue. Halting execution.")
        return None, None

    print(f"✅ Using GCP Project: {project_id}")

    # --- EDIT: Ask for the bucket name from the user ---
    base_bucket_name = input("Please enter a base name for your GCS bucket (e.g., 'bitcoin-data'): ").strip() # Added .strip()
    if not base_bucket_name:
        print("🔴 ERROR: A base name for the bucket is required. Halting execution.")
        return project_id, None

    # GCS bucket names must be globally unique and cannot contain dots or start with underscores.
    # We create a safe, unique name by combining the project ID and the user's input.
    safe_project_id = project_id.replace('.', '-')
    bucket_name = f"{safe_project_id}-{base_bucket_name}"
    print(f"✅ Bucket will be named: {bucket_name}")
    return project_id, bucket_name

# Run the setup function and store the variables.
PROJECT_ID, BUCKET_NAME = setup_environment()

# --- NEW: Ensure PROJECT_ID and BUCKET_NAME are not None before setting env vars ---
if PROJECT_ID is None or BUCKET_NAME is None:
    print("🔴 CRITICAL ERROR: PROJECT_ID or BUCKET_NAME was not set. Please re-run this cell and provide the required inputs.")
else:
    os.environ["PROJECT_ID"] = PROJECT_ID
    os.environ["BUCKET_NAME"] = BUCKET_NAME

Authenticating your Google account...
✅ Authentication successful.
⚠️ Could not automatically determine GCP Project ID.
Please enter your GCP Project ID: mgmt467-project1
✅ Using GCP Project: mgmt467-project1
Please enter a base name for your GCS bucket (e.g., 'bitcoin-data'): mgmt479-bitcoin-final-lab
✅ Bucket will be named: mgmt467-project1-mgmt479-bitcoin-final-lab


In [2]:
# @title ### Cell 2: Create GCS Bucket (Corrected)
# @markdown **Objective:** This cell checks if the required Google Cloud Storage (GCS) bucket exists, and if not, it creates it.

def create_gcs_bucket_if_not_exists(bucket_name, project_id):
    """
    Checks for the existence of a GCS bucket and creates it if it's not found.

    This function uses the `gsutil` command-line tool to interact with GCS.
    It first tries to list the contents of the target bucket. If this command
    fails, it assumes the bucket does not exist and proceeds to create it
    using `gsutil mb` (make bucket).

    Args:
        bucket_name (str): The name of the GCS bucket to check and create.
        project_id (str): The GCP project ID to associate with the bucket creation.
    """
    if not bucket_name or not project_id:
        print("🔴 ERROR: Bucket name or Project ID is not set. Cannot proceed.")
        return

    print(f"\nChecking for GCS bucket: gs://{bucket_name}")
    try:
        # The `gsutil ls` command will fail if the bucket does not exist.
        subprocess.run(["gsutil", "ls", f"gs://{bucket_name}"], check=True, capture_output=True)
        print(f"✅ Bucket gs://{bucket_name} already exists.")
    except subprocess.CalledProcessError:
        print(f"Bucket not found. Creating gs://{bucket_name}...")
        # `gsutil mb` creates a new bucket.
        try:
            # --- FIX: Added the '-p [project_id]' flag to the gsutil command ---
            subprocess.run(
                ["gsutil", "mb", "-p", project_id, f"gs://{bucket_name}"],
                check=True, capture_output=True, text=True
            )
            print(f"✅ Bucket gs://{bucket_name} created.")
        except subprocess.CalledProcessError as e:
            print(f"🔴 ERROR: Failed to create bucket. The name may be taken or invalid.")
            print(f"   Details: {e.stderr}")


# Run the bucket creation function using the globally defined variables.
create_gcs_bucket_if_not_exists(BUCKET_NAME, PROJECT_ID)


Checking for GCS bucket: gs://mgmt467-project1-mgmt479-bitcoin-final-lab
✅ Bucket gs://mgmt467-project1-mgmt479-bitcoin-final-lab already exists.


In [3]:
import os
import subprocess
import re
import pandas as pd # Added pandas for potential future data manipulation

def process_and_upload_btc_data(local_file_path, bucket_name):
    """
    Processes the local BTC USD 1-min data CSV and uploads it to GCS.

    This function performs the following steps:
    1. Reads the local CSV file.
    2. Performs basic cleaning (e.g., removing quotes, ensuring consistent line endings).
    3. Uploads the cleaned CSV file to a specific folder within the GCS bucket.
    4. Cleans up any temporary local files (though for this scenario, the original is kept).

    Args:
        local_file_path (str): The path to the local BTC USD 1-min data CSV file.
        bucket_name (str): The GCS bucket to upload the final data to.
    """
    print(f"\n--- Processing and uploading data from {local_file_path} ---")

    cleaned_csv_file = "cleaned_btcusd_1-min_data.csv" # New name for the cleaned file

    # 1. Read and clean the CSV data
    print("  Cleaning...")
    try:
        with open(local_file_path, 'r', encoding='utf-8', errors='ignore') as infile, \
             open(cleaned_csv_file, 'w', encoding='utf-8') as outfile:
            for line in infile:
                # Apply similar cleaning logic: remove trailing commas, remove quotes
                cleaned_line = re.sub(r',?$', '', line.strip()).replace('"', '')
                outfile.write(cleaned_line + '\n')
        print(f"  ✅ Successfully cleaned data to {cleaned_csv_file}")
    except Exception as e:
        print(f"  🔴 ERROR: Failed to clean local CSV file. {e}")
        return

    # 2. Upload the cleaned file to Google Cloud Storage
    print("  Storing in GCS...")
    destination_blob_name = f"data/bitcoinETL/{cleaned_csv_file}" # New folder for BTC data
    try:
        subprocess.run(
            ["gsutil", "cp", cleaned_csv_file, f"gs://{bucket_name}/{destination_blob_name}"],
            check=True, capture_output=True
        )
        print(f"  ✅ Successfully uploaded {cleaned_csv_file} to GCS.")
    except subprocess.CalledProcessError as e:
        print(f"  🔴 ERROR: Failed to upload to GCS. {e.stderr.decode()}")

    # 3. Clean up the temporary cleaned file
    print("  Cleaning up temporary local files...")
    if os.path.exists(cleaned_csv_file):
        os.remove(cleaned_csv_file)
    print("  Cleanup complete.")

# Define the local file path for the bitcoin data
BTC_DATA_LOCAL_PATH = "/content/btcusd_1-min_data.csv"

# Now, call the new function
process_and_upload_btc_data(BTC_DATA_LOCAL_PATH, BUCKET_NAME)


--- Processing and uploading data from /content/btcusd_1-min_data.csv ---
  Cleaning...
  🔴 ERROR: Failed to clean local CSV file. [Errno 2] No such file or directory: '/content/btcusd_1-min_data.csv'


In [4]:
import pandas as pd
import subprocess
import os # Ensure os is imported for os.getenv

def load_cleaned_btc_data_from_gcs(bucket_name_arg):
    """
    Loads the cleaned BTC USD 1-min data from GCS into a pandas DataFrame.

    Args:
        bucket_name_arg (str): The GCS bucket where the cleaned data is stored.

    Returns:
        pd.DataFrame: A pandas DataFrame containing the cleaned BTC data.
    """
    # Prioritize fetching from environment, then use argument
    actual_bucket_name = os.getenv("BUCKET_NAME", bucket_name_arg)
    if not actual_bucket_name:
        print("🔴 ERROR: Bucket name is not available. Cannot proceed.")
        return pd.DataFrame()

    print(f"\n--- Loading cleaned BTC data from GCS: gs://{actual_bucket_name}/data/bitcoinETL/cleaned_btcusd_1-min_data.csv ---")
    gcs_path = f"gs://{actual_bucket_name}/data/bitcoinETL/cleaned_btcusd_1-min_data.csv"
    local_temp_file = "temp_cleaned_btcusd.csv"

    try:
        # Download the file from GCS to a local temporary file
        subprocess.run(["gsutil", "cp", gcs_path, local_temp_file], check=True, capture_output=True)
        print(f"  ✅ Successfully downloaded {gcs_path} to {local_temp_file}.")

        # Specify column names for robust loading, assuming 9 columns as per common format
        column_names = ['Unix_Timestamp', 'Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Volume_BTC', 'Volume_USD']

        # Load the local temporary file into a pandas DataFrame
        # Use skiprows=1 to skip the header row
        df = pd.read_csv(local_temp_file, header=None, names=column_names, skiprows=1)
        print("  ✅ Successfully loaded data into pandas DataFrame (pre-conversion).")

        # Fix 'Symbol' column: if it contains numeric-like values, replace with 'BTCUSD'
        # This handles cases where the raw data might have a corrupted Symbol column.
        df['Symbol'] = 'BTCUSD'

        # Convert 'Unix_Timestamp' to int64, coercing errors to NaN and then dropping
        df['Unix_Timestamp'] = pd.to_numeric(df['Unix_Timestamp'], errors='coerce')
        df.dropna(subset=['Unix_Timestamp'], inplace=True)
        df['Unix_Timestamp'] = df['Unix_Timestamp'].astype('int64')

        # Convert 'Unix_Timestamp' to 'Date' for accuracy, as direct 'Date' parsing seems problematic
        df['Date'] = pd.to_datetime(df['Unix_Timestamp'], unit='s', errors='coerce')

        # Convert numeric columns, coercing errors
        numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume_BTC', 'Volume_USD']
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        print("  ✅ Successfully converted data types.")

        # Clean up the local temporary file
        os.remove(local_temp_file)
        print("  ✅ Cleaned up local temporary file.")

        # Check for unparseable dates (which will now be NaT from Unix_Timestamp conversion)
        if df['Date'].isnull().any():
            print("  ⚠️ Warning: Some 'Unix_Timestamp' entries were unparseable and converted to NaT in 'Date' column.")
            if df['Date'].isnull().sum() / len(df) > 0.5: # More than 50% NaT
                print("  🔴 CRITICAL WARNING: More than 50% of 'Date' entries are NaT. Unix Timestamp conversion likely incorrect.")

        return df

    except subprocess.CalledProcessError as e:
        print(f"  🔴 ERROR: Failed to download or load data from GCS. {e.stderr.decode()}")
        return pd.DataFrame()
    except Exception as e:
        print(f"  🔴 ERROR: An unexpected error occurred: {e}")
        return pd.DataFrame()

# Load the cleaned data into a DataFrame
btc_df = load_cleaned_btc_data_from_gcs(BUCKET_NAME)

# Display the first few rows of the DataFrame
if not btc_df.empty:
    print("\nFirst 5 rows of the loaded DataFrame:")
    display(btc_df.head())
else:
    print("\nDataFrame is empty, possibly due to an error during loading or all dates being unparseable.")


--- Loading cleaned BTC data from GCS: gs://mgmt467-project1-mgmt479-bitcoin-final-lab/data/bitcoinETL/cleaned_btcusd_1-min_data.csv ---
  ✅ Successfully downloaded gs://mgmt467-project1-mgmt479-bitcoin-final-lab/data/bitcoinETL/cleaned_btcusd_1-min_data.csv to temp_cleaned_btcusd.csv.
  ✅ Successfully loaded data into pandas DataFrame (pre-conversion).
  ✅ Successfully converted data types.
  ✅ Cleaned up local temporary file.

First 5 rows of the loaded DataFrame:


,Unix_Timestamp,Date,Symbol,Open,High,Low,Close,Volume_BTC,Volume_USD
0,1325412060,2012-01-01 10:01:00,BTCUSD,4.58,4.58,0.0,NaN,NaN,NaN
1,1325412120,2012-01-01 10:02:00,BTCUSD,4.58,4.58,0.0,NaN,NaN,NaN
2,1325412180,2012-01-01 10:03:00,BTCUSD,4.58,4.58,0.0,NaN,NaN,NaN
3,1325412240,2012-01-01 10:04:00,BTCUSD,4.58,4.58,0.0,NaN,NaN,NaN
4,1325412300,2012-01-01 10:05:00,BTCUSD,4.58,4.58,0.0,NaN,NaN,NaN


In [5]:
import os
from google.colab import auth
from google.cloud import bigquery

# Authenticate to Google
auth.authenticate_user()

# Set up project and dataset details
PROJECT_ID = os.getenv("PROJECT_ID")      # your project (already defined from setup_environment)
REGION     = "us-central1" # Corrected region from 'us-central'

# Update BIGQUERY_DATASET and TABLE_PATH for BTC data
BIGQUERY_DATASET = "bitcoin_data_set"
CLEANED_VIEW = "bitcoin_cleaned"
TABLE_PATH = f"{PROJECT_ID}.{BIGQUERY_DATASET}.{CLEANED_VIEW}"

os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["REGION"]     = REGION

# Create BigQuery client
bq = bigquery.Client(project=PROJECT_ID)

print("BQ Project:", PROJECT_ID)
print("BigQuery Dataset:", BIGQUERY_DATASET)
print("BigQuery Table Path:", TABLE_PATH)

# --- NEW: Create BigQuery Dataset if it doesn't exist and verify ---
try:
    dataset_ref = bq.dataset(BIGQUERY_DATASET, project=PROJECT_ID)
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = REGION  # Set the geographic location for the dataset

    # Attempt to create the dataset
    bq.create_dataset(dataset, exists_ok=True)  # API request
    print(f"✅ BigQuery Dataset '{BIGQUERY_DATASET}' creation attempt finished.")

    # Verify dataset existence after creation attempt
    bq.get_dataset(dataset_ref) # This will raise NotFound if it doesn't exist
    print(f"✅ BigQuery Dataset '{BIGQUERY_DATASET}' confirmed to exist in {REGION}.")
except Exception as e:
    print(f"🔴 ERROR: Failed to create or verify BigQuery Dataset '{BIGQUERY_DATASET}'. Details: {e}")

BQ Project: mgmt467-project1
BigQuery Dataset: bitcoin_data_set
BigQuery Table Path: mgmt467-project1.bitcoin_data_set.bitcoin_cleaned
✅ BigQuery Dataset 'bitcoin_data_set' creation attempt finished.
✅ BigQuery Dataset 'bitcoin_data_set' confirmed to exist in us-central1.


In [6]:
# @title ### Cell 4: Main Execution Loop
# @markdown **Objective:** This cell defines the date range for the data download and then loops through each month, calling the worker function defined in the previous cell.

import pandas as pd
from google.cloud import bigquery
from datetime import datetime
import os # For os.environ, though not directly used in the functions below, it's often present.


# Assume btc_df, PROJECT_ID, BIGQUERY_DATASET, bq client are already defined globally from previous cells.

def process_bitcoin_data_for_month(year, month, btc_dataframe, project_id, bigquery_dataset, cleaned_view, bq_client):
    """
    Filters the global BTC DataFrame for a specific month and loads it into BigQuery.

    Args:
        year (int): The year for which to process data.
        month (int): The month for which to process data.
        btc_dataframe (pd.DataFrame): The DataFrame containing all BTC data.
        project_id (str): The GCP Project ID.
        bigquery_dataset (str): The BigQuery dataset name.
        cleaned_view (str): The BigQuery table name for cleaned data.
        bq_client (bigquery.Client): The BigQuery client object.
    """
    print(f"\n  Processing data for {year}-{month:02d}...")

    # Ensure 'Date' column is clean before filtering
    # Drop rows where 'Date' is NaT (Not a Time) or if the Date is outside standard range
    # For robust filtering, we make sure 'Date' column exists and is datetime type.
    if 'Date' not in btc_dataframe.columns or not pd.api.types.is_datetime64_any_dtype(btc_dataframe['Date']):
        print("  🔴 ERROR: 'Date' column is missing or not in datetime format. Skipping.")
        return

    clean_df = btc_dataframe.dropna(subset=['Date']).copy()

    # Filter data for the specific month
    monthly_df = clean_df[
        (clean_df['Date'].dt.year == year) &
        (clean_df['Date'].dt.month == month)
    ]

    if monthly_df.empty:
        print(f"  No data found for {year}-{month:02d}. Skipping BigQuery load.")
        return

    # Define the BigQuery table ID
    # This will append to the main table 'bitcoin_cleaned'
    table_id = f"{project_id}.{bigquery_dataset}.{cleaned_view}"

    # Prepare data for BigQuery (ensure column names and types match)
    # Convert Unix_Timestamp to integer (INT64 in BigQuery)
    monthly_df['Unix_Timestamp'] = monthly_df['Unix_Timestamp'].astype('int64')

    # Load data into BigQuery
    try:
        # Use bigquery.Client from the global scope (initialized in NO-llDhEM2QX)
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND" # Append to the table if it exists
        )
        job = bq_client.load_table_from_dataframe(
            monthly_df, table_id, job_config=job_config
        )
        job.result() # Wait for the job to complete
        print(f"  ✅ Successfully loaded {len(monthly_df)} rows for {year}-{month:02d} to BigQuery table {table_id}.")
    except Exception as e:
        print(f"  🔴 ERROR: Failed to load data for {year}-{month:02d} to BigQuery. Details: {e}")

def run_pipeline(project_id, bucket_name, btc_dataframe, bigquery_dataset, cleaned_view, bq_client):
    """
    Manages the overall execution flow. It sets the date range and iterates
    through each month, calling the bitcoin-specific processing function.
    """
    if not bucket_name:
        print("🔴 ERROR: BUCKET_NAME is not defined. Halting execution.")
        return
    if btc_dataframe.empty: # Check if btc_dataframe was loaded successfully
        print("🔴 ERROR: BTC DataFrame is empty. Cannot process monthly data.")
        return

    # --- Define Date Range ---
    # You can change these values to process a different range of data.
    # IMPORTANT: The actual data in 'btcusd_1-min_data.csv' needs to cover this range.
    # The current 'btcusd_1-min_data.csv' likely contains data from earlier years.
    # Adjust start_year/month and end_year/month to match the data present in btc_df.
    start_year, start_month = 2012, 1  # Start: January 2012 (Adjusted to match data)
    end_year, end_month = 2012, 3      # End: March 2012 (Adjusted to match data)

    print(f"Starting pipeline to process bitcoin data from {start_month}/{start_year} to {end_month}/{end_year}.")

    # --- Loop Through Dates ---
    current_date = datetime(start_year, start_month, 1)
    end_date = datetime(end_year, end_month, 1)

    while current_date <= end_date:
        # Call the bitcoin-specific processing function
        process_bitcoin_data_for_month(current_date.year, current_date.month, btc_dataframe,
                                       project_id, bigquery_dataset, cleaned_view, bq_client)

        # Increment the date to the next month
        # This logic correctly handles rolling over to the next year.
        next_month = current_date.month + 1
        next_year = current_date.year
        if next_month > 12:
            next_month = 1
            next_year += 1
        current_date = datetime(next_year, next_month, 1)

    print("\n--- Pipeline Execution Complete ---")

# --- Streamlined dependencies: relying on global variables from preceding cells ---
# All necessary variables (PROJECT_ID, BUCKET_NAME, btc_df, BIGQUERY_DATASET, CLEANED_VIEW, bq)
# are expected to be globally defined and correctly populated by the execution of previous cells.

print("--- Environment Setup for this cell (streamlined) complete. ---")

# Execute the main pipeline function.
run_pipeline(PROJECT_ID, BUCKET_NAME, btc_df, BIGQUERY_DATASET, CLEANED_VIEW, bq)

--- Environment Setup for this cell (streamlined) complete. ---
Starting pipeline to process bitcoin data from 1/2012 to 3/2012.

  Processing data for 2012-01...


/tmp/ipython-input-370920220.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_df['Unix_Timestamp'] = monthly_df['Unix_Timestamp'].astype('int64')


  ✅ Successfully loaded 44039 rows for 2012-01 to BigQuery table mgmt467-project1.bitcoin_data_set.bitcoin_cleaned.

  Processing data for 2012-02...


/tmp/ipython-input-370920220.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_df['Unix_Timestamp'] = monthly_df['Unix_Timestamp'].astype('int64')


  ✅ Successfully loaded 41760 rows for 2012-02 to BigQuery table mgmt467-project1.bitcoin_data_set.bitcoin_cleaned.

  Processing data for 2012-03...


/tmp/ipython-input-370920220.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_df['Unix_Timestamp'] = monthly_df['Unix_Timestamp'].astype('int64')


  ✅ Successfully loaded 44640 rows for 2012-03 to BigQuery table mgmt467-project1.bitcoin_data_set.bitcoin_cleaned.

--- Pipeline Execution Complete ---


In [7]:
# @title ### Cell 5: Final Verification
# @markdown **Objective:** This final cell runs verification steps to ensure data was successfully processed and stored.

def verify_uploads_and_bigquery(bucket_name, project_id, bigquery_dataset, cleaned_view):
    """
    Lists the contents of the target GCS directory and queries BigQuery to verify uploads.
    """
    if not bucket_name:
        print("🔴 ERROR: BUCKET_NAME is not defined. Cannot verify GCS uploads.")
    else:
        target_directory_gcs = f"gs://{bucket_name}/data/bitcoinETL/" # Corrected directory for bitcoin data
        print(f"\nVerifying final contents of GCS directory: {target_directory_gcs}...")

        try:
            # Use subprocess to run the gsutil command and capture its output.
            result = subprocess.run(
                ["gsutil", "ls", "-l", target_directory_gcs],
                check=True, capture_output=True, text=True
            )
            print("✅ GCS Uploads confirmed:")
            print(result.stdout)
        except subprocess.CalledProcessError as e:
            print(f"🔴 ERROR: Could not list GCS directory contents. It may be empty or there was an error.")
            print(e.stderr)

    # Verify data in BigQuery
    if not project_id or not bigquery_dataset or not cleaned_view:
        print("🔴 ERROR: BigQuery project, dataset, or view name is not defined. Cannot verify BigQuery data.")
    else:
        print(f"\nVerifying data in BigQuery table: {project_id}.{bigquery_dataset}.{cleaned_view}...")
        try:
            # Ensure bq client is initialized (it should be from earlier cells)
            from google.cloud import bigquery
            bq_client = bigquery.Client(project=project_id)

            query = f"""SELECT COUNT(*) FROM `{project_id}.{bigquery_dataset}.{cleaned_view}`"""
            query_job = bq_client.query(query)
            results = query_job.result()

            for row in results:
                row_count = row[0]
                print(f"✅ BigQuery verification successful: Table contains {row_count} rows.")
        except Exception as e:
            print(f"🔴 ERROR: Failed to query BigQuery table for verification. Details: {e}")

# Run the verification function using the globally defined variables.
# These variables should be available from previous cells.
verify_uploads_and_bigquery(BUCKET_NAME, PROJECT_ID, BIGQUERY_DATASET, CLEANED_VIEW)



Verifying final contents of GCS directory: gs://mgmt467-project1-mgmt479-bitcoin-final-lab/data/bitcoinETL/...
✅ GCS Uploads confirmed:
 384706218  2025-12-13T21:09:33Z  gs://mgmt467-project1-mgmt479-bitcoin-final-lab/data/bitcoinETL/cleaned_btcusd_1-min_data.csv
TOTAL: 1 objects, 384706218 bytes (366.88 MiB)


Verifying data in BigQuery table: mgmt467-project1.bitcoin_data_set.bitcoin_cleaned...
✅ BigQuery verification successful: Table contains 260878 rows.


In [8]:
# @title ### Cell 6: Create BigQuery Dataset
# @markdown **Objective:** This cell creates a new dataset in BigQuery to store our flight data. A dataset is a container for your tables, similar to a schema in a traditional database.

from google.cloud import bigquery
from google.cloud.exceptions import Conflict

# Initialize the BigQuery client
client = bigquery.Client(project=PROJECT_ID)

# Define the name for your new BigQuery dataset
BIGQUERY_DATASET = "bitcoin_data_set"
dataset_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}"

try:
    # Create a Dataset object
    dataset = bigquery.Dataset(dataset_id)
    # Specify the location for the dataset
    dataset.location = "US" # You can change this to your preferred location
    # Make an API request to create the dataset
    client.create_dataset(dataset, timeout=30)
    print(f"✅ Successfully created dataset: {dataset_id}")
except Conflict:
    print(f"✅ Dataset '{dataset_id}' already exists.")
except Exception as e:
    print(f"🔴 An error occurred: {e}")

✅ Dataset 'mgmt467-project1.bitcoin_data_set' already exists.


In [9]:
# @title ### Cell 7: Load All CSVs from GCS into a BigQuery Table
# @markdown **Objective:** This cell uses a BigQuery Load Job to efficiently load all the cleaned CSV files from your GCS bucket into a single BigQuery table. This is the recommended method for batch loading from GCS.

# Define the name for the new table
BIGQUERY_TABLE_FULL_LOAD = "bitcoin_full_dataset" # New table name for the full bitcoin dataset
table_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}.{BIGQUERY_TABLE_FULL_LOAD}"

# Configure the Load Job
job_config = bigquery.LoadJobConfig(
    # Automatically infer the schema from the data.
    autodetect=True,
    # Skip the first row of each file, which contains the headers.
    skip_leading_rows=1,
    # The source format is CSV.
    source_format=bigquery.SourceFormat.CSV,
    # Allow for rows that might have too few columns.
    allow_jagged_rows=True,
    # Overwrite the table if it already exists.
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

# Define the GCS URI pointing to the single cleaned BTC data file
uri = f"gs://{BUCKET_NAME}/data/bitcoinETL/cleaned_btcusd_1-min_data.csv"

try:
    # Start the Load Job
    load_job = bq.load_table_from_uri(
        uri, table_id, job_config=job_config
    )
    print(f"🚀 Starting BigQuery load job {load_job.job_id}...")

    # Waits for the job to complete.
    load_job.result()
    print("✅ Load job finished.")

    # Get the destination table object and print the row count
    destination_table = bq.get_table(table_id)
    print(f"Loaded {destination_table.num_rows} rows into table '{BIGQUERY_TABLE_FULL_LOAD}'.")

except Exception as e:
    print(f"🔴 An error occurred: {e}")

🚀 Starting BigQuery load job 554fd8da-8635-4b23-a936-0da67dbf957f...
✅ Load job finished.
Loaded 7335037 rows into table 'bitcoin_full_dataset'.


In [10]:
# @title ### Cell 8: Data Cleaning and Feature Engineering (Corrected)
# @markdown **Objective:** This cell runs a SQL query to create a new, cleaned view of our data. This view will serve as the basis for our machine learning models.

# Define the name for our new analytics view
BITCOIN_ANALYTICS_VIEW = "bitcoin_analytics_view" # New view name for bitcoin analysis
view_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}.{BITCOIN_ANALYTICS_VIEW}"

# --- Debugging: Print BigQuery table schema ---
print(f"\n--- Inspecting BigQuery table schema for {table_id} ---")
try:
    table = client.get_table(table_id)  # API request
    print("Schema for `bitcoin_full_dataset`:")
    for field in table.schema:
        print(f"  - {field.name}: {field.field_type}")
except Exception as e:
    print(f"🔴 ERROR: Could not retrieve schema for {table_id}. Details: {e}")
print("---------------------------------------------------")
# --- End Debugging ---

# This SQL query selects relevant columns and creates our new feature.
# A VIEW is a virtual table based on the result-set of an SQL statement.
# It's a great way to create a clean dataset without duplicating data.
sql_query = f"""
CREATE OR REPLACE VIEW `{view_id}` AS
SELECT
  Timestamp, -- Corrected column name based on schema inspection
  FORMAT_TIMESTAMP('%Y-%m-%d %H:%M:%S', TIMESTAMP_SECONDS(CAST(Timestamp AS INT64))) AS datetime, -- Cast Timestamp to INT64
  Open,
  High,
  Low,
  Close,
  Volume, -- Corrected column name based on schema inspection
  -- Create a boolean label: TRUE if Close price is higher than Open price for that minute.
  CASE
    WHEN Close > Open THEN TRUE
    ELSE FALSE
  END AS is_positive_minute_change
FROM
  `{table_id}` -- Use the full bitcoin dataset table (created in Cell 7)
WHERE
  Open IS NOT NULL AND Close IS NOT NULL -- Filter out entries where Open or Close prices are missing
"""

try:
    # Execute the query to create the view
    query_job = client.query(sql_query)
    query_job.result() # Wait for the job to complete
    print(f"✅ Successfully created analytics view: {BITCOIN_ANALYTICS_VIEW}")

    # Verify by showing the first 10 rows of the new view
    print("\n--- Sample of Bitcoin Analytics Data ---")
    df_analytics = client.query(f"SELECT * FROM `{view_id}` LIMIT 10").to_dataframe()
    display(df_analytics)

except Exception as e:
    print(f"🔴 An error occurred: {e}")


--- Inspecting BigQuery table schema for mgmt467-project1.bitcoin_data_set.bitcoin_full_dataset ---
Schema for `bitcoin_full_dataset`:
  - Timestamp: FLOAT
  - Open: FLOAT
  - High: FLOAT
  - Low: FLOAT
  - Close: FLOAT
  - Volume: FLOAT
---------------------------------------------------
✅ Successfully created analytics view: bitcoin_analytics_view

--- Sample of Bitcoin Analytics Data ---


,Timestamp,datetime,Open,High,Low,Close,Volume,is_positive_minute_change
0,1.329603e+09,2012-02-18 22:09:00,4.14,4.14,4.14,4.14,0.0,False
1,1.329601e+09,2012-02-18 21:39:00,4.14,4.14,4.14,4.14,0.0,False
2,1.329602e+09,2012-02-18 21:56:00,4.14,4.14,4.14,4.14,0.0,False
3,1.329603e+09,2012-02-18 22:10:00,4.14,4.14,4.14,4.14,0.0,False
4,1.329604e+09,2012-02-18 22:32:00,4.14,4.14,4.14,4.14,0.0,False
5,1.329603e+09,2012-02-18 22:11:00,4.14,4.14,4.14,4.14,0.0,False
6,1.329604e+09,2012-02-18 22:27:00,4.14,4.14,4.14,4.14,0.0,False
7,1.329601e+09,2012-02-18 21:33:00,4.14,4.14,4.14,4.14,0.0,False
8,1.329601e+09,2012-02-18 21:30:00,4.14,4.14,4.14,4.14,0.0,False
9,1.329434e+09,2012-02-16 23:11:00,4.15,4.15,4.15,4.15,0.0,False


In [11]:
# @title ### Cell 9: Build, Evaluate, and Predict with a Linear Regression Model
# @markdown **Objective:** Use BigQuery ML to create a linear regression model to predict the Bitcoin Close price.

from google.cloud import bigquery
from google.cloud.exceptions import NotFound # Import NotFound for specific error handling

# Re-initialize the BigQuery client to ensure it's fresh and correctly scoped
client = bigquery.Client(project=PROJECT_ID)

# Re-define necessary variables as they might have been cleared
BITCOIN_ANALYTICS_VIEW = "bitcoin_analytics_view"
view_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}.{BITCOIN_ANALYTICS_VIEW}"

print(f"Attempting to use view: {view_id}")

# --- Add a check for view existence before proceeding ---
try:
    client.get_table(view_id) # get_table works for views too
    print(f"✅ BigQuery view '{view_id}' confirmed to exist.")
except NotFound:
    print(f"🔴 ERROR: BigQuery view '{view_id}' not found. Please ensure Cell 8 (Data Cleaning and Feature Engineering) ran successfully and created the view.")
    raise # Re-raise the error to stop execution if the view is truly not found
except Exception as e:
    print(f"🔴 An unexpected error occurred while checking view existence: {e}")
    raise

# --- 1. Create the Linear Regression Model ---
print("🚀 Training Linear Regression model...")
create_linear_model_query = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{BIGQUERY_DATASET}.bitcoin_price_predictor`
OPTIONS(model_type='LINEAR_REG', input_label_cols=['Close']) AS
SELECT
  Close,
  Open,
  High,
  Low
  -- Volume -- Excluded due to nulls impacting prediction data
FROM
  `{view_id}`;
"""
linear_job = client.query(create_linear_model_query)
linear_job.result()
print("✅ Linear Regression model created successfully.")

# --- 2. Evaluate the Model ---
print("\n--- Model Evaluation ---")
evaluate_linear_model_query = f"""
SELECT * FROM ML.EVALUATE(MODEL `{PROJECT_ID}.{BIGQUERY_DATASET}.bitcoin_price_predictor`);
"""
linear_eval_df = client.query(evaluate_linear_model_query).to_dataframe()
display(linear_eval_df)

# --- Debugging Prediction Input Data ---
print("\n--- Debugging Prediction Input Data --- ")
debug_query = f"""
SELECT COUNT(*)
FROM
  `{view_id}`
WHERE
  Open IS NOT NULL AND NOT IS_NAN(Open) AND
  High IS NOT NULL AND NOT IS_NAN(High) AND
  Low IS NOT NULL AND NOT IS_NAN(Low);
"""
debug_df = client.query(debug_query).to_dataframe()
print(f"Number of rows satisfying prediction input criteria (excluding Volume): {debug_df.iloc[0,0]}")

# --- 3. Make Predictions with the Model ---
print("\n--- Sample Predictions ---")
predict_linear_query = f"""
WITH PredictionInput AS (
  SELECT
    Timestamp,
    datetime, -- Include datetime for robust join
    Close, -- Include Close for actual_close comparison
    Open,
    High,
    Low
    -- Volume -- Excluded due to nulls impacting prediction data
  FROM
    `{view_id}`
  WHERE
    Open IS NOT NULL AND NOT IS_NAN(Open) AND
    High IS NOT NULL AND NOT IS_NAN(High) AND
    Low IS NOT NULL AND NOT IS_NAN(Low)
    -- Volume IS NOT NULL AND NOT IS_NAN(Volume) -- Excluded due to nulls impacting prediction data
  LIMIT 10 -- Limiting input data for sample predictions
)
SELECT
  pi.Close AS actual_close,
  p.predicted_Close
FROM
  ML.PREDICT(MODEL `{PROJECT_ID}.{BIGQUERY_DATASET}.bitcoin_price_predictor`,
    -- Pass only features to the model for prediction
    (SELECT Timestamp, datetime, Open, High, Low FROM PredictionInput) -- Exclude Volume
  ) AS p
JOIN
  PredictionInput AS pi
ON
  p.datetime = pi.datetime -- Join on datetime string
ORDER BY pi.Timestamp
"""
linear_predict_df = client.query(predict_linear_query).to_dataframe()
display(linear_predict_df)

Attempting to use view: mgmt467-project1.bitcoin_data_set.bitcoin_analytics_view
✅ BigQuery view 'mgmt467-project1.bitcoin_data_set.bitcoin_analytics_view' confirmed to exist.
🚀 Training Linear Regression model...
✅ Linear Regression model created successfully.

--- Model Evaluation ---


,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,4.101975,110.782597,9.986267e-07,0.629642,1.0,1.0



--- Debugging Prediction Input Data --- 
Number of rows satisfying prediction input criteria (excluding Volume): 7335037

--- Sample Predictions ---


,actual_close,predicted_Close


In [12]:
# @title ### Cell 10: Build, Evaluate, and Predict with a Logistic Regression Model
# @markdown **Objective:** Use BigQuery ML to create a logistic regression model to predict if Bitcoin's close price will be higher than its open price for a given minute.

# --- 1. Create the Logistic Regression Model ---
print("🚀 Training Logistic Regression model...")
create_logistic_model_query = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{BIGQUERY_DATASET}.bitcoin_price_change_classifier`
OPTIONS(model_type='LOGISTIC_REG', input_label_cols=['is_positive_minute_change']) AS
SELECT
  is_positive_minute_change,
  Open,
  High,
  Low
  -- Volume -- Excluded due to nulls impacting prediction data
FROM
  `{view_id}`;
"""
logistic_job = client.query(create_logistic_model_query)
logistic_job.result()
print("✅ Logistic Regression model created successfully.")

# --- 2. Evaluate the Model ---
print("\n--- Model Evaluation ---")
evaluate_logistic_model_query = f"""
SELECT * FROM ML.EVALUATE(MODEL `{PROJECT_ID}.{BIGQUERY_DATASET}.bitcoin_price_change_classifier`);
"""
logistic_eval_df = client.query(evaluate_logistic_model_query).to_dataframe()
display(logistic_eval_df)

# --- Debugging Prediction Input Data ---
print("\n--- Debugging Prediction Input Data (Logistic) ---")
debug_logistic_query = f"""
SELECT COUNT(*)
FROM
  `{view_id}`
WHERE
  Open IS NOT NULL AND NOT IS_NAN(Open) AND
  High IS NOT NULL AND NOT IS_NAN(High) AND
  Low IS NOT NULL AND NOT IS_NAN(Low);
"""
debug_logistic_df = client.query(debug_logistic_query).to_dataframe()
print(f"Number of rows satisfying logistic prediction input criteria: {debug_logistic_df.iloc[0,0]}")


# --- 3. Make Predictions with the Model ---
print("\n--- Sample Predictions ---")
predict_logistic_query = f"""
SELECT
  is_positive_minute_change AS actual_is_positive_change,
  predicted_is_positive_minute_change,
  predicted_is_positive_minute_change_probs
FROM
  ML.PREDICT(MODEL `{PROJECT_ID}.{BIGQUERY_DATASET}.bitcoin_price_change_classifier`,
    (SELECT
       Timestamp,
       datetime,
       is_positive_minute_change,
       Open,
       High,
       Low
     FROM
       `{view_id}`
     WHERE
       Open IS NOT NULL AND NOT IS_NAN(Open) AND
       High IS NOT NULL AND NOT IS_NAN(High) AND
       Low IS NOT NULL AND NOT IS_NAN(Low)
     ORDER BY Timestamp
     LIMIT 10
    )
  )
ORDER BY Timestamp;
"""
logistic_predict_df = client.query(predict_logistic_query).to_dataframe()
display(logistic_predict_df)

🚀 Training Logistic Regression model...
✅ Logistic Regression model created successfully.

--- Model Evaluation ---


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.418052,0.105106,0.650005,0.167979,0.62325,0.683479



--- Debugging Prediction Input Data (Logistic) ---
Number of rows satisfying logistic prediction input criteria: 7335037

--- Sample Predictions ---


,actual_is_positive_change,predicted_is_positive_minute_change,predicted_is_positive_minute_change_probs
0,False,False,"[{'label': True, 'prob': 0.28593256093076536},..."
1,False,False,"[{'label': True, 'prob': 0.28593256093076536},..."
2,False,False,"[{'label': True, 'prob': 0.28593256093076536},..."
3,False,False,"[{'label': True, 'prob': 0.28593256093076536},..."
4,False,False,"[{'label': True, 'prob': 0.28593256093076536},..."
5,False,False,"[{'label': True, 'prob': 0.28593256093076536},..."
6,False,False,"[{'label': True, 'prob': 0.28593256093076536},..."
7,False,False,"[{'label': True, 'prob': 0.28593256093076536},..."
8,False,False,"[{'label': True, 'prob': 0.28593256093076536},..."
9,False,False,"[{'label': True, 'prob': 0.28593256093076536},..."


In [13]:
from google.cloud import bigquery
from IPython.display import display
from google.colab import auth
import os

# Re-authenticate to ensure fresh credentials
auth.authenticate_user()

# Ensure necessary variables are available (assuming they are from preceding cells)
# PROJECT_ID, BIGQUERY_DATASET, BITCOIN_ANALYTICS_VIEW, linear_eval_df, logistic_eval_df
# You might need to adjust these if your Colab session has reset and previous cells haven't run.

print("\n--- Bitcoin Data Analytics Summary ---")

# Initialize the BigQuery client (if not already in global scope)
if 'client' not in globals() or client.project != PROJECT_ID:
    client = bigquery.Client(project=PROJECT_ID)

# Define the view ID for bitcoin analytics
BITCOIN_ANALYTICS_VIEW = "bitcoin_analytics_view"
view_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}.{BITCOIN_ANALYTICS_VIEW}"

print(f"\n--- Sample of Bitcoin Analytics Data from View: {view_id} ---")
try:
    df_analytics_sample = client.query(f"SELECT * FROM `{view_id}` LIMIT 10").to_dataframe()
    display(df_analytics_sample)
except Exception as e:
    print(f"🔴 ERROR: Could not retrieve sample from '{view_id}'. Details: {e}")

print("\n--- Linear Regression Model Evaluation (bitcoin_price_predictor) ---")
# Ensure linear_eval_df is available, or re-run evaluation if needed
if 'linear_eval_df' in globals():
    display(linear_eval_df)
else:
    print("🔴 linear_eval_df not found. Please ensure Cell 9 executed successfully.")

print("\n--- Logistic Regression Model Evaluation (bitcoin_price_change_classifier) ---")
# Ensure logistic_eval_df is available, or re-run evaluation if needed
if 'logistic_eval_df' in globals():
    display(logistic_eval_df)
else:
    print("🔴 logistic_eval_df not found. Please ensure Cell 10 executed successfully.")

print("\n--- How to view in BigQuery Console ---")
print(f"To view your BigQuery data, models, and views in the Google Cloud Console:")
print(f"1. Navigate to: https://console.cloud.google.com/bigquery")
print(f"2. Ensure your project '{PROJECT_ID}' is selected.")
print(f"3. In the explorer pane, expand '{BIGQUERY_DATASET}' to see your tables (e.g., 'bitcoin_full_dataset', 'bitcoin_cleaned') and views (e.g., 'bitcoin_analytics_view').")
print(f"4. Expand 'Models' to see your trained models (e.g., 'bitcoin_price_predictor', 'bitcoin_price_change_classifier').")



--- Bitcoin Data Analytics Summary ---

--- Sample of Bitcoin Analytics Data from View: mgmt467-project1.bitcoin_data_set.bitcoin_analytics_view ---


,Timestamp,datetime,Open,High,Low,Close,Volume,is_positive_minute_change
0,1.329603e+09,2012-02-18 22:09:00,4.14,4.14,4.14,4.14,0.0,False
1,1.329601e+09,2012-02-18 21:39:00,4.14,4.14,4.14,4.14,0.0,False
2,1.329602e+09,2012-02-18 21:56:00,4.14,4.14,4.14,4.14,0.0,False
3,1.329603e+09,2012-02-18 22:10:00,4.14,4.14,4.14,4.14,0.0,False
4,1.329604e+09,2012-02-18 22:32:00,4.14,4.14,4.14,4.14,0.0,False
5,1.329603e+09,2012-02-18 22:11:00,4.14,4.14,4.14,4.14,0.0,False
6,1.329604e+09,2012-02-18 22:27:00,4.14,4.14,4.14,4.14,0.0,False
7,1.329601e+09,2012-02-18 21:33:00,4.14,4.14,4.14,4.14,0.0,False
8,1.329601e+09,2012-02-18 21:30:00,4.14,4.14,4.14,4.14,0.0,False
9,1.329434e+09,2012-02-16 23:11:00,4.15,4.15,4.15,4.15,0.0,False



--- Linear Regression Model Evaluation (bitcoin_price_predictor) ---


,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,4.101975,110.782597,9.986267e-07,0.629642,1.0,1.0



--- Logistic Regression Model Evaluation (bitcoin_price_change_classifier) ---


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.418052,0.105106,0.650005,0.167979,0.62325,0.683479



--- How to view in BigQuery Console ---
To view your BigQuery data, models, and views in the Google Cloud Console:
1. Navigate to: https://console.cloud.google.com/bigquery
2. Ensure your project 'mgmt467-project1' is selected.
3. In the explorer pane, expand 'bitcoin_data_set' to see your tables (e.g., 'bitcoin_full_dataset', 'bitcoin_cleaned') and views (e.g., 'bitcoin_analytics_view').
4. Expand 'Models' to see your trained models (e.g., 'bitcoin_price_predictor', 'bitcoin_price_change_classifier').


In [14]:
# @title ### Cell 11: Build and Analyze a K-Means Clustering Model
# @markdown **Objective:** Use BigQuery ML's K-Means to group Bitcoin data into distinct clusters based on price characteristics.

# --- 1. Create the K-Means Clustering Model ---
print("🚀 Training K-Means Clustering model...")
create_kmeans_model_query = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{BIGQUERY_DATASET}.bitcoin_clusterer`
OPTIONS(model_type='KMEANS', num_clusters=5) AS
SELECT
  Open,
  High,
  Low,
  Close,
  Volume
FROM
  `{view_id}`
WHERE
  Open IS NOT NULL AND High IS NOT NULL AND Low IS NOT NULL AND Close IS NOT NULL AND Volume IS NOT NULL;
"""
kmeans_job = client.query(create_kmeans_model_query)
kmeans_job.result()
print("✅ K-Means Clustering model created successfully.")

# --- 2. Analyze the Cluster Centroids ---
print("\n--- Cluster Centroid Analysis ---")
# ML.CENTROIDS shows the average values for each feature within each cluster.
# This helps us understand what defines each cluster.
analyze_centroids_query = f"""
SELECT
  * EXCEPT(centroid_id)
FROM
  ML.CENTROIDS(MODEL `{PROJECT_ID}.{BIGQUERY_DATASET}.bitcoin_clusterer`);
"""
centroids_df = client.query(analyze_centroids_query).to_dataframe()
display(centroids_df)

# --- 3. See which Cluster Each Bitcoin Minute Belongs To ---
print("\n--- Sample Cluster Assignments ---")
predict_kmeans_query = f"""
SELECT
  CENTROID_ID,
  Timestamp, -- Include Timestamp for identification
  datetime,  -- Include datetime for identification
  Open,
  High,
  Low,
  Close,
  Volume
FROM
  ML.PREDICT(MODEL `{PROJECT_ID}.{BIGQUERY_DATASET}.bitcoin_clusterer`,
    (SELECT Timestamp, datetime, Open, High, Low, Close, Volume FROM `{view_id}` LIMIT 10));
"""
kmeans_predict_df = client.query(predict_kmeans_query).to_dataframe()
display(kmeans_predict_df)


🚀 Training K-Means Clustering model...
✅ K-Means Clustering model created successfully.

--- Cluster Centroid Analysis ---


,feature,numerical_value,categorical_value
0,Open,3133.547198,[]
1,High,3143.588168,[]
2,Low,3120.671559,[]
3,Close,3131.988707,[]
4,Volume,153.826579,[]
5,Open,55831.175832,[]
6,High,55852.491191,[]
7,Low,55809.543270,[]
8,Close,55831.464293,[]
9,Volume,1.814244,[]



--- Sample Cluster Assignments ---


,CENTROID_ID,Timestamp,datetime,Open,High,Low,Close,Volume
0,5,1.329601e+09,2012-02-18 21:35:00,4.14,4.14,4.14,4.14,0.0
1,5,1.329383e+09,2012-02-16 09:06:00,4.17,4.17,4.17,4.17,0.0
2,5,1.329603e+09,2012-02-18 22:14:00,4.14,4.14,4.14,4.14,0.0
3,5,1.329382e+09,2012-02-16 08:43:00,4.17,4.17,4.17,4.17,0.0
4,5,1.329600e+09,2012-02-18 21:24:00,4.14,4.14,4.14,4.14,0.0
5,5,1.329604e+09,2012-02-18 22:29:00,4.14,4.14,4.14,4.14,0.0
6,5,1.329377e+09,2012-02-16 07:28:00,4.17,4.17,4.17,4.17,0.0
7,5,1.328055e+09,2012-02-01 00:08:00,3.80,3.80,3.80,3.80,0.0
8,5,1.329434e+09,2012-02-16 23:21:00,4.15,4.15,4.15,4.15,0.0
9,5,1.329380e+09,2012-02-16 08:19:00,4.17,4.17,4.17,4.17,0.0
